# Constraining Large Language Models with Guidance
In this notebook, we're going to learn how to use Guidance, a programming paradigm that offers superior control and efficiency compared to conventional prompting and chaining

In [2]:
!pip install guidance


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [203]:
import guidance
from guidance import models, gen, select, user, assistant, system, substring

In [204]:
model_path = "models/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
llm = models.LlamaCpp(model_path, n_gpu_layers=1)

## Detecting emotion 🔍🤔🧐
Let's start by writing a prompt to detect the emotion in pieces of text.

In [205]:
def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()      

In [206]:
text = read_file("data/linkedin.txt")
print(text)

Woohoo, 🎁's starting to come in early this year--just got the acceptance note for my session "Data Contracts In Practice With Debezium and Apache Flink" for #KafkaSummit London '24 🇬🇧. See you there in March!


In [207]:
question = "What is the emotion of the following text and how strong is the emotion on a scale from 1-100?"
lm = llm + f"{question}: {text}\n"
lm += "Answer: " + gen(name='answer')

## Restrict to set of options 🔒📋✅ 
We can restrict the text generated using the `select` function 

In [208]:
emotions = [
    'happy', 'sad', 'angry', 'surprised', 
    'disgusted', 'excited', 'fearful', 'neutral'
]  

In [209]:
lm = llm + f"{question}: {text}\n"
lm += "Answer: " + select(emotions, name='emotion')

## Regular expressions 🔍🧩🔤
We can pass the argument `regex` to the `gen` function to control what gets generated.

In [210]:
lm = llm + f"{question}: {text}\n"
lm += "Answer: " + select(emotions, name='emotion') 
lm += ", Scale: " + gen(regex='\d+', name='strength')

In [211]:
lm['emotion'], lm['strength']

('happy', '90')

## Reusable Components ♻️🧩🔁
We can put all that code into a function to make it easier to detect the emotion of new pieces of text.

In [212]:
@guidance
def emotion_detector(lm, text):
    question = "What is the emotion of the following text and how strong is the emotion on a scale from 1-100?"
    emotions = [
        'happy', 'sad', 'angry', 'surprised', 
        'disgusted', 'excited', 'fearful', 'neutral'
    ]
    lm = llm + f"{question}: {text}\n"
    lm += "Answer: " + select(emotions, name='emotion') 
    lm += ", Scale: " + gen(regex='\d+', name='strength')
    return lm  

In [213]:
jack_text = read_file("data/jack.txt")
print(jack_text)

A last-16 US Open finish and a first ATP Tour final have made 2023 Jack Draper's most successful year of his young career so far. But for the 21-year-old Briton, success has been tinged by sadness with his grandmother Brenda - a former tennis player and coach - unable to recognise his achievements. Draper's grandmother has Alzheimer's disease, a condition that causes dementia and the gradual decline of cognitive functioning in the brain.


In [214]:
llm + emotion_detector(jack_text)

In [215]:
llm + emotion_detector(read_file("data/novak.txt"))

## Returning JSON 🔄📄🔡 
Guidance can also generate output in a JSON format, which is useful for connecting with other tools.

In [216]:
@guidance
def emotion_detector_json(lm, text):
    question = "What is the emotion of the following text and how strong is the emotion on a scale from 1-100?"
    emotions = [
        'happy', 'sad', 'angry', 'surprised', 
        'disgusted', 'excited', 'fearful', 'neutral'
    ]
    lm = llm + f"{question}: {text}\n"
    only_numbers_pattern = r'\d+'
    lm += f"""{{
        "text": "{text}",
        "emotion": "{select(emotions, name='answer')}",
        "scale": {gen(regex=only_numbers_pattern, name='strength')}
    }}"""
    return lm

In [217]:
llm + emotion_detector_json(read_file("data/dyche.txt"))